In [10]:
#pip install nbformat

In [81]:
import torch
import torchvision
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import torchvision.transforms as T
print(torch.__version__)

2.0.1


In [82]:
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image

In [83]:
from pathlib import Path
current_directory = Path.cwd()

# Get the parent directory (one level up)
parent_directory = current_directory.parent
    
    
path_to_dataset =str(parent_directory)+'/captcha_dataset'

captchas = os.listdir(path_to_dataset)

Y = []
X = []
# png's are 3D and 4D but all dimension are the same execpt for the 4th which is all balck
# only turn png into array and keep the first dimension
for i,img_name in enumerate(captchas):
    Y.append(img_name[:5])

    new_path = path_to_dataset + '/' + captchas[i]

    image = Image.open(new_path)

    array_image = np.array(image) 

    X.append(array_image[:,:,0])

  

In [5]:
import string

all_characters= list(string.ascii_lowercase) + list(string.digits)

print(all_characters)

nchar = len(all_characters)

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


In [6]:
# make label array

y = np.zeros((5,len(Y),nchar)) #5*1070*36(5 letters in captcha) with all entries 0

for i,captcha in enumerate(Y):
   
  temp_label =np.zeros((5,nchar))

  for j, character in enumerate(captcha):
     
      character_index =  all_characters.index(character)
      temp_label[j,character_index] = 1
  y[:,i] = temp_label

In [ ]:
n = 1
rand_i = np.random.randint(0,len(Y),size=(n))
#362

In [45]:
def show_digit(pic,a,b,label,ind):
    plt.imshow(pic[:,a:b],cmap='gray')

    plt.title(label[ind])
    plt.show()


In [ ]:
rand_i = np.random.randint(0,len(Y))

pic = X[rand_i][:,30:150]
plt.imshow(pic)
plt.show()
label = Y[rand_i] 
diffs=[0]

large_characters=['w','m']
small_characters = ['f','f','1','l','t']

small_size= 15
normal_size=25
large_size= 40
for i in range(5):
    
    if label[i] in large_characters:

        show_digit(pic,sum(diffs),sum(diffs)+large_size,label,i)

        diffs.append(large_size)
    
    elif str(label[i]) in small_characters:

        show_digit(pic,sum(diffs),sum(diffs)+small_size,label,i)
        diffs.append(small_size)

    else:
        show_digit(pic,sum(diffs),sum(diffs)+normal_size,label,i)
        diffs.append(normal_size)

In [ ]:
# CNN Model
import torch.nn as nn

class CustomModel(nn.Module):
    def __init__(self, imgshape, nchar):
        super(CustomModel, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=(3, 3), padding=(1, 1))
        self.relu1 = nn.ReLU()
        self.mp1 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(1, 1))

        self.conv2 = nn.Conv2d(16, 32, kernel_size=(3, 3), padding=(1, 1))
        self.relu2 = nn.ReLU()
        self.mp2 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(1, 1))

        self.conv3 = nn.Conv2d(32, 32, kernel_size=(3, 3), padding=(1, 1))
        self.relu3 = nn.ReLU()
        self.mp3 = nn.MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=(1, 1))

        self.flatten = nn.Flatten()

        self.dens1 = nn.Linear(32 * 7 * 25, 64) # Calculate the input size based on the shapes

        self.dropout = nn.Dropout(0.5)
        self.dens_out = nn.Linear(64, nchar)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu1(x)
        x = self.mp1(x)

        x = self.conv2(x)
        x = self.relu2(x)
        x = self.mp2(x)

        x = self.conv3(x)
        x = self.relu3(x)
        x = self.mp3(x)

        x = self.flatten(x)
        x = self.dens1(x)
        x = self.relu3(x)

        outs = []
        for _ in range(5):
            x_drop = self.dropout(x)
            out = self.dens_out(x_drop)
            outs.append(out)

        return outs

# Define the input shape and number of characters in the output
imgshape = (1, 50, 200)
nchar = 5  # Replace 10 with the actual number of characters in the output (e.g., the number of classes in the classification task)

# Create the model
model = CustomModel(imgshape, nchar)


In [ ]:
import torch.optim as optim

loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

EMNIST MODEL
<br>
https://machinelearningmastery.com/building-a-convolutional-neural-network-in-pytorch/

In [87]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import EMNIST


class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(in_features=128 * 3 * 3, out_features=512)  # Adjusted input size
        self.fc2 = nn.Linear(in_features=512, out_features=256)
        self.fc3 = nn.Linear(in_features=256, out_features=62)  # 62 classes (10 digits + 52 letters)
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = self.pool(torch.relu(self.conv3(x)))
        x = x.view(-1, 128 * 3 * 3)  # Reshape before fully connected layers
        x = self.dropout(torch.relu(self.fc1(x)))
        x = self.dropout(torch.relu(self.fc2(x)))
        x = self.fc3(x)
        return x



model = CNN()

In [ ]:
# Load and preprocess EMNIST data
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
train_dataset = EMNIST(root='./data', split='byclass', train=True, transform=transform, download=True)
test_dataset = EMNIST(root='./data', split='byclass', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [88]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

In [ ]:
# Training loop
num_epochs = 1
for epoch in range(num_epochs):
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Evaluate the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in test_loader:
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on test data: {100 * correct / total:.2f}%')